In [ ]:
## IMPORT MODULE TO PREPROCESS THE QUESTIONS

from preprocess_questions import *

In [ ]:
path_train = "data/train.csv" # XX : to fill
path_test = "data/test.csv" # XX : to fill

In [ ]:
## PREPROCESS QUESTIONS AND SAVE TO train_processed AND test_processed

main(path_train, path_test)

In [ ]:
## IMPORT MODULE TO PERFORM FEATURE EXTRACTION

from generate_features import *

In [ ]:
path_train = "train_processed.csv"
path_test = "test_processed.csv"

df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

df_train['q1p'] = df_train.q1p.progress_apply(eval)
df_train['q2p'] = df_train.q2p.progress_apply(eval)
df_test['q1p'] = df_test.q1p.progress_apply(eval)
df_test['q2p'] = df_test.q2p.progress_apply(eval)

In [ ]:
## INSTANTIATE A FEATURE EXTRACTOR

feature_extractor = FeatureExtractor([TfidfVectorizer(ngram_range = (1,3) ,stop_words='english'),
                                          CountVectorizer(ngram_range = (1,3) ,stop_words='english')])

In [ ]:
## FIT THE FEATURE EXTRACTOR

feature_extractor.fit(df_train, df_test)

In [ ]:
## EXTRACT FEATURES AND CREATE FINAL DATASET

import time
t = time.time()
x_train, x_test = feature_extractor.transform(df_train, df_test)
print(' ')
print(' Done in %d seconds' % (int(time.time() - t)))

In [ ]:
## SAVE THE FEATURES

from sklearn.externals import joblib

joblib.dump(x_train, 'dataset_train.pkl')
joblib.dump(x_test, 'dataset_test.pkl')

In [ ]:
y_train = df_train.is_duplicate.values

In [ ]:
## MODULE TO TRAIN THE LGBM MODEL

from lgbm_model import *

In [ ]:
## TRAIN THE LGBM MODEL

train_lgbm()

In [ ]:
## NEXT CELLS : EXTRACT NLP AND NON NLP FEATURES FOR
## THE AHMET'S LSTM MODEL AND TRAIN THE LSTM MODEL

In [ ]:
!python lstm_nlp.py

In [ ]:
!python lstm_non_nlp.py

In [ ]:
!python lstm_model.py

In [ ]:
## AVERAGE THE 10 OUT OF FOLDS PREDICTIONS FOR EACH MODEL

path = "lgbm_preds"
scores_lgbm = np.zeros((20179,))
for i in range(10):
    scores_lgbm += pd.read_csv(path + str(i) + ".csv").is_duplicate.values
    
scores_lgbm /= 10

path = "lstm_preds"
scores_lstm = np.zeros((20179,))
for i in range(10):
    scores_lstm += pd.read_csv(path + str(i) + ".csv").is_duplicate.values
    
scores_lstm /= 10

In [ ]:
## MODULE TO POSTPROCESS SCORES

from postprocess import *

In [ ]:
## BOOST SCORES FOR EACH MODEL

post_process(scores_lgbm, "10folds_lgbm_boost_2.csv")
post_process(scores_lstm, "10folds_lstm_boost_2.csv")

In [ ]:
## WEIGHTED AVERAGE OF THE BOOSTED SCORES OF EACH MODEL

preds_lgbm = pd.read_csv("10folds_lgbm_boost_2.csv")
preds_lstm = pd.read_csv("10folds_lstm_boost_2.csv")

preds = 0.6 * preds_lstm['Score'].values + 0.4 * preds_lgbm['Score'].values

In [ ]:
## BOOST 1x THE PREDICTIONS AND SAVE THE FILE TO SUBMIT ON KAGGLE
post_process(preds, 'final_submission.csv', REPEAT = 1)